<a href="https://colab.research.google.com/github/Yan-Weijia/CompLing/blob/master/HW0402.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install deeppavlov

In [0]:
!pip install tensorflow==1.15.0

In [0]:
!apt-get --yes install git

In [0]:
!pip install git+https://github.com/deepmipt/bert.git@feat/multi_gpu

In [0]:
import pandas as pd

In [0]:
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/pristavki.csv

In [0]:
data = pd.read_csv('pristavki.csv', header=None, names=['text'])

In [0]:
!wget https://raw.githubusercontent.com/Yan-Weijia/CompLing/master/HW04data/test.txt
!wget https://raw.githubusercontent.com/Yan-Weijia/CompLing/master/HW04data/train.txt
!wget https://raw.githubusercontent.com/Yan-Weijia/CompLing/master/HW04data/valid.txt

In [31]:
!ls *.txt

test.txt  train.txt  valid.txt


In [32]:
!head test.txt

Brother B-PRS
In I-PRS
Arms: I-PRS
Earned I-PRS
In I-PRS
Blood I-PRS
- O
продолжение O
отменного O
экшна O


In [0]:
import json
from deeppavlov import configs, build_model, train_model

In [0]:
with configs.ner.ner_ontonotes_bert_mult.open(encoding='utf8') as f:
    ner_config = json.load(f)

ner_config['dataset_reader']['data_path'] = './'  # directory with train.txt, valid.txt and test.txt files
ner_config['metadata']['variables']['NER_PATH'] = './'
ner_config['metadata']['download'] = [ner_config['metadata']['download'][-1]]  # do not download the pretrained ontonotes model

ner_model = train_model(ner_config, download=True)

In [35]:
ner_model(['Brother In Arms: Earned In Blood', 'Brother In Arms: Road To Hill 30'])

[[['Brother', 'In', 'Arms', ':', 'Earned', 'In', 'Blood'],
  ['Brother', 'In', 'Arms', ':', 'Road', 'To', 'Hill', '30']],
 [['B-PRS', 'I-PRS', 'I-PRS', 'O', 'B-PRS', 'I-PRS', 'I-PRS'],
  ['B-PRS', 'I-PRS', 'I-PRS', 'I-PRS', 'B-PRS', 'I-PRS', 'I-PRS', 'I-PRS']]]

In [36]:
ner_model(['fifa 17', 'фифа 15, 16', 'гта 5', 'GTA V', 'видмак 3'])

[[['fifa', '17'],
  ['фифа', '15', ',', '16'],
  ['гта', '5'],
  ['GTA', 'V'],
  ['видмак', '3']],
 [['B-PRS', 'I-PRS'],
  ['B-PRS', 'I-PRS', 'O', 'I-PRS'],
  ['B-PRS', 'I-PRS'],
  ['B-PRS', 'I-PRS'],
  ['B-PRS', 'I-PRS']]]

In [37]:
ner_model(['Street Fighter IV', 'Grand Theft Auto 4', 'Call of Duty: Black Ops III', 'Gold of Wars'])

[[['Street', 'Fighter', 'IV'],
  ['Grand', 'Theft', 'Auto', '4'],
  ['Call', 'of', 'Duty', ':', 'Black', 'Ops', 'III'],
  ['Gold', 'of', 'Wars']],
 [['B-PRS', 'I-PRS', 'I-PRS'],
  ['B-PRS', 'I-PRS', 'I-PRS', 'I-PRS'],
  ['B-PRS', 'I-PRS', 'I-PRS', 'I-PRS', 'B-PRS', 'I-PRS', 'I-PRS'],
  ['B-PRS', 'I-PRS', 'I-PRS']]]

In [0]:
marked = []

for text in data.text.values[:1000]:
    # BERT имеет лимит на длину текста в 512 слов, возьмем даже еще меньше
    if len(text.split()) > 100:
        continue
    pred = ner_model([text])
    sent, tags = pred[0][0], pred[1][0]
    
    # достанем только тексты с сущностями
    if len(set(tags[0])) > 1:
        marked.append(list(zip(sent,tags)))

In [43]:
marked[0]

[('Battlefield', 'B-PRS'),
 ('4', 'I-PRS'),
 (',', 'O'),
 ('Granturismo', 'B-PRS'),
 ('3', 'I-PRS'),
 ('и', 'O'),
 ('Звезды', 'O'),
 ('Playstation', 'O'),
 ('Битва', 'O'),
 ('сильнейших', 'O'),
 ('.', 'O'),
 ('За', 'O'),
 ('все', 'O'),
 ('1500', 'O'),
 ('руб', 'O'),
 ('.', 'O')]

Не очень умно опознавает версии после "name" + ":"